# 3.6. Tratamiento de series temporales II.

In [ ]:
import pandas as pd
import numpy  as np

## Time Series con indices duplicados

- Podemos tener fechas duplicadas en el índice:

In [ ]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

In [ ]:
dup_ts.index.is_unique

In [ ]:
dup_ts['1/3/2000']  # not duplicated

In [ ]:
dup_ts['1/2/2000']  # duplicated

- Para elminarlos podemos agrupar por nivel 0 y realizar una operación que los unifique

In [ ]:
grouped = dup_ts.groupby(level=0)
grouped.mean()

In [ ]:
grouped.first()

In [ ]:
grouped.count()

### Date Offsets

In [ ]:
from pandas.tseries.offsets import Hour, Minute

In [ ]:
rango = pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')
rango

In [ ]:
rango + Hour(2) + Minute(30)

#### Desplazamiento

- Existen dos métodos principales: ``shift()`` and ``tshift()``
 * ``shift()`` desplaza los datos 
 * ``tshift()`` desplaza el índice.
-En ambos casos, se especifica por la frecuencia.

In [ ]:
dates = pd.date_range('2015-07-25', periods=15, freq='B')
data = pd.DataFrame({'close':[10,12,14,15,15,19, 20,17, 15, 14, 12,13,13,14,10]}, index=dates)
data

In [ ]:
data.shift(5)

In [ ]:
data.tshift(5)

## Remuestreo
- La indexación de Pandas permite operaciones de remuestreo con relativa facilidad. 
- Es común necesitar obtener los datos a mayor o menor frecuencia de la disponible.
- Para ello se puede usar ``resample()`` o ``asfreq()`` (más sencillo).
- La diferencia principal es que mientras la primera es una agregación de datos, la segunda es una selección de los mismos. 

In [ ]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')
ts = pd.DataFrame(np.random.randn(len(rng)), index=rng)
ts

In [ ]:
ts.resample('M').mean()

In [ ]:
ts.resample('BM').mean()

- Mientras que ``resample()`` devuelve la media del periodo
- ``asfreq()`` devuelve el último valor (omitiendo el mes en caso de no existir).
- Una ventaja adicional de ``asfreq()`` es la capacidad de imputar valores.

In [ ]:
ts.asfreq('5D')

### Downsampling

In [ ]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')
ts = pd.Series(np.arange(12), index=rng)
ts

In [ ]:
ts.resample('5min').sum()

In [ ]:
ts.resample('5min', closed='right').sum()

In [ ]:
ts.resample('5min', closed='right', label='right').sum()

### Open-High-Low-Close (OHLC) resampling

In [ ]:
ts

In [ ]:
ts.resample('5min').ohlc()

### Upsampling and Interpolation

In [ ]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

In [ ]:
df_daily = frame.resample('D').asfreq()
df_daily

In [ ]:
frame.resample('D').ffill()

In [ ]:
frame.resample('D').ffill(limit=2)

## Ventanas móviles

- Se utiliza la función ``rolling()``, que funciona de forma similar a ``groupby``
- Con distintas operaciones de agregación disponibles.

In [ ]:
close_px_all = pd.read_csv('examples/stock_px_2.csv',
                           parse_dates=True, index_col=0)
close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
close_px

In [ ]:
close_px = close_px.resample('B').ffill

In [ ]:
close_px.AAPL.plot()
close_px.AAPL.rolling(250).mean().plot()

In [ ]:
appl_std250 = close_px.AAPL.rolling(250, min_periods=10).std()
appl_std250[5:12]

In [ ]:
appl_std250.plot()

In [ ]:
expanding_mean = appl_std250.expanding().mean()

In [ ]:
expanding_mean

In [ ]:
close_px.rolling(60).mean().plot(logy=True)

In [ ]:
close_px.rolling('20D').mean()

### Exponentially Weighted

In [ ]:
aapl_px = close_px.AAPL['2006':'2007']
ma60 = aapl_px.rolling(30, min_periods=20).mean()
ewma60 = aapl_px.ewm(span=30).mean()
ma60.plot(style='k--', label='Simple MA')
ewma60.plot(style='k-', label='EW MA')

### Binary Moving Window

In [ ]:
spx_px = close_px_all['SPX']
spx_rets = spx_px.pct_change()
returns = close_px.pct_change()

In [ ]:
corr = returns.AAPL.rolling(125, min_periods=100).corr(spx_rets)
corr.plot()

In [ ]:
plt.figure()

In [ ]:
corr = returns.rolling(125, min_periods=100).corr(spx_rets)
corr.plot()

### Moving Window definadas por el usuario

In [ ]:
from scipy.stats import percentileofscore
score_at_2percent = lambda x: percentileofscore(x, 0.02)
result = returns.AAPL.rolling(250).apply(score_at_2percent)

In [ ]:
result.plot()